In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import time

def scrape_presidency_only():
    # --- [1. 날짜 판별기 함수 (핵심 로직)] ---
    def is_president_term(date_obj):
        # 1기 임기: 2017.01.20 ~ 2021.01.20
        term1_start = datetime(2017, 1, 20)
        term1_end = datetime(2021, 1, 20)
        
        # 2기 임기: 2025.01.20 ~ 현재 (2029년 예정)
        term2_start = datetime(2025, 1, 20)
        
        # 날짜가 1기 또는 2기 사이에 있는지 확인
        if term1_start <= date_obj <= term1_end:
            return True
        if date_obj >= term2_start:
            return True
        
        return False

    # --- [2. 브라우저 설정] ---
    url = "https://rollcall.com/factbase/trump/search/"
    opt = webdriver.ChromeOptions()
    opt.add_argument("--headless=new") 
    opt.add_argument("--no-sandbox")
    opt.add_argument("--disable-dev-shm-usage")
    opt.add_argument("--window-size=1920,1080")
    opt.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opt)
    
    results = []
    
    try:
        print(f"🌐 접속 중: {url}")
        driver.get(url)
        time.sleep(5)

        # --- [3. 무한 스크롤 & 실시간 필터링] ---
        last_height = driver.execute_script("return document.body.scrollHeight")
        scroll_count = 0
        max_scrolls = 300 # 최대 스크롤 횟수
        
        print("⬇️ 스크롤을 내리며 임기 기간 데이터를 찾습니다...")
        
        processed_urls = set() # 중복 방지용

        while scroll_count < max_scrolls:
            # HTML 파싱
            soup = BeautifulSoup(driver.page_source, "html.parser")
            
            # 제목(날짜) & 링크 찾기
            title_divs = soup.select('div[x-text="item.record_title"]')
            link_btns = soup.select('a[title="View Transcript"]')
            
            min_len = min(len(title_divs), len(link_btns))
            
            for i in range(min_len):
                # A. URL 가져오기
                link = link_btns[i].get('href')
                if not link: continue
                if "rollcall.com" not in link: link = "https://rollcall.com" + link
                
                # 이미 처리한 URL이면 패스 (성능 향상)
                if link in processed_urls:
                    continue
                
                processed_urls.add(link)

                # B. 날짜 추출 및 필터링
                full_text = title_divs[i].text.strip()
                
                if " - " in full_text:
                    date_str = full_text.split(" - ")[-1].strip()
                    try:
                        # 문자열("January 21, 2026") -> 날짜 객체로 변환
                        dt_obj = datetime.strptime(date_str, "%B %d, %Y")
                        
                        # [검문소] 대통령 임기 기간인가?
                        if is_president_term(dt_obj):
                            # 합격! 저장
                            results.append({
                                'date': date_str,
                                'url': link,
                                'title': full_text.split(" - ")[0].strip()
                            })
                            # print(f"  ✅ [수집] {date_str}") # 로그 너무 많으면 주석 처리
                        else:
                            # 불합격! (야인 시절 등)
                            pass
                            # print(f"  🚫 [제외] {date_str} (임기 아님)")

                    except ValueError:
                        pass # 날짜 형식이 이상하면 건너뜀
            
            # 스크롤 다운
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                print("🎉 바닥 도달!")
                break
            last_height = new_height
            scroll_count += 1
            
            # 중간 점검 출력 (10번마다)
            if scroll_count % 10 == 0:
                print(f"   ... 스크롤 {scroll_count}회 | 수집된 데이터: {len(results)}건")

    except Exception as e:
        print(f"❌ 에러: {e}")

    finally:
        driver.quit()
        
    # --- [4. CSV 저장] ---
    if results:
        df = pd.DataFrame(results)
        # 날짜순 정렬 (최신순)
        filename = "trump_presidency_only.csv"
        df.to_csv(filename, index=False, encoding="utf-8-sig")
        print(f"\n💾 저장 완료: '{filename}' (총 {len(df)}건)")
        return df
    else:
        print("⚠️ 조건에 맞는 데이터가 없습니다.")
        return None

# 실행
df_presidency = scrape_presidency_only()
if df_presidency is not None:
    print(df_presidency.head())

🌐 접속 중: https://rollcall.com/factbase/trump/search/
⬇️ 스크롤을 내리며 임기 기간 데이터를 찾습니다...
   ... 스크롤 10회 | 수집된 데이터: 397건
   ... 스크롤 20회 | 수집된 데이터: 792건
   ... 스크롤 30회 | 수집된 데이터: 976건
   ... 스크롤 40회 | 수집된 데이터: 976건
   ... 스크롤 50회 | 수집된 데이터: 1175건
   ... 스크롤 60회 | 수집된 데이터: 1575건
   ... 스크롤 70회 | 수집된 데이터: 1974건
   ... 스크롤 80회 | 수집된 데이터: 2374건
   ... 스크롤 90회 | 수집된 데이터: 2773건
   ... 스크롤 100회 | 수집된 데이터: 3173건
   ... 스크롤 110회 | 수집된 데이터: 3483건
